<a href="https://colab.research.google.com/github/aaditya1012/fraud-detection-model/blob/main/Fraud_Detection_Model_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn==1.6.1


In [2]:
from google.colab import files

# Open file upload dialog
uploaded = files.upload()



Saving Synthetic_Financial_datasets_log.csv to Synthetic_Financial_datasets_log.csv


In [3]:
import pandas as pd

# Load dataset from /content/
df = pd.read_csv("/content/Synthetic_Financial_datasets_log.csv")

# Display first few rows
df.head()


step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0

In [5]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

# 📌 1. Load Dataset
df = pd.read_csv('/content/Synthetic_Financial_datasets_log.csv')  # Ensure this CSV is in the same folder

# 📌 2. Encode Categorical Columns
encoder = LabelEncoder()
df['type'] = encoder.fit_transform(df['type'])

# 📌 3. Drop Unnecessary Columns
df = df.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'])

# 📌 4. Split Features and Target
X = df.drop(columns=['isFraud'])
y = df['isFraud']

# 📌 5. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 📌 6. Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 📌 7. Apply SMOTE for Imbalanced Data
smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 📌 8. Train Model
model_balanced = RandomForestClassifier(n_estimators=100, random_state=42)
model_balanced.fit(X_resampled, y_resampled)

# 📌 9. Evaluate Model
y_pred_balanced = model_balanced.predict(X_test)
print("✅ Model Accuracy:", accuracy_score(y_test, y_pred_balanced))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred_balanced))

# 📌 10. Save Model for FastAPI (scikit-learn 1.6.1)
with open("fraud_detection_model.pkl", "wb") as file:
    pickle.dump(model_balanced, file)

print("✅ Model saved as fraud_detection_model.pkl (Trained with scikit-learn 1.6.1)")


✅ Model Accuracy: 0.9992565955534041

✅ Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.64      0.96      0.77      1643

    accuracy                           1.00   1272524
   macro avg       0.82      0.98      0.88   1272524
weighted avg       1.00      1.00      1.00   1272524

✅ Model saved as fraud_detection_model.pkl (Trained with scikit-learn 1.6.1)


In [7]:
from google.colab import files

# Download the trained model
files.download("fraud_detection_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
print(X.columns)  # This will show all feature names used during training


Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest'],
      dtype='object')
